In [1]:
import pandas as pd
import numpy as np
dataset=pd.read_csv('data.csv')
import re
import ast
import json
import nltk
from nltk.stem.porter import PorterStemmer
import pickle

In [2]:
dataset.head()

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


1)Unique_id will remain as it is.
2)crawl_timestamp??
3)product_url will fetch the product 
4)Product name as it is
5)Product Category tree is the main thing which we are going to use for tagging
6)pid and unique id difference??
7)retail price to be displayed as it is and display in ecommerce fashion along with discounted price.>>tagging?
8)image good
9)isfkadvantage?? >> can be used for tagging
10)productrating and overallrating >>content and collaborative approach both
11)brand as it is>> used for tagging
13 specification dekhlo ek baar kya karna he 

In [3]:
dataset['product_specifications'][0]

'{"product_specification"=>[{"key"=>"Number of Contents in Sales Package", "value"=>"Pack of 3"}, {"key"=>"Fabric", "value"=>"Cotton Lycra"}, {"key"=>"Type", "value"=>"Cycling Shorts"}, {"key"=>"Pattern", "value"=>"Solid"}, {"key"=>"Ideal For", "value"=>"Women\'s"}, {"value"=>"Gentle Machine Wash in Lukewarm Water, Do Not Bleach"}, {"key"=>"Style Code", "value"=>"ALTHT_3P_21"}, {"value"=>"3 shorts"}]}'

In [4]:
#build model then make cleaner version of your code
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   uniq_id                  20000 non-null  object 
 1   crawl_timestamp          20000 non-null  object 
 2   product_url              20000 non-null  object 
 3   product_name             20000 non-null  object 
 4   product_category_tree    20000 non-null  object 
 5   pid                      20000 non-null  object 
 6   retail_price             19922 non-null  float64
 7   discounted_price         19922 non-null  float64
 8   image                    19997 non-null  object 
 9   is_FK_Advantage_product  20000 non-null  bool   
 10  description              19998 non-null  object 
 11  product_rating           20000 non-null  object 
 12  overall_rating           20000 non-null  object 
 13  brand                    14136 non-null  object 
 14  product_specifications

In [5]:
dataset.isnull().sum()

uniq_id                       0
crawl_timestamp               0
product_url                   0
product_name                  0
product_category_tree         0
pid                           0
retail_price                 78
discounted_price             78
image                         3
is_FK_Advantage_product       0
description                   2
product_rating                0
overall_rating                0
brand                      5864
product_specifications       14
dtype: int64

In [6]:
dataset=dataset.dropna()

In [7]:
dataset.isnull().sum()

uniq_id                    0
crawl_timestamp            0
product_url                0
product_name               0
product_category_tree      0
pid                        0
retail_price               0
discounted_price           0
image                      0
is_FK_Advantage_product    0
description                0
product_rating             0
overall_rating             0
brand                      0
product_specifications     0
dtype: int64

In [8]:
dataset.shape

(14058, 15)

In [9]:
dataset.drop(['crawl_timestamp', 'is_FK_Advantage_product', 'product_rating', 'overall_rating'], axis=1, inplace=True)


In [10]:
dataset.head()

,uniq_id,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,description,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [11]:
dataset.drop(['uniq_id'], axis=1, inplace=True)

In [12]:
dataset.iloc[0].description

"Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts"

In [13]:
dataset.iloc[1].description

"FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Price: Rs. 22,646 • Fine deep seating experience • Save Space with the all new click clack Sofa Bed • Easy to fold and vice versa with simple click clack mechanism • Chrome legs with mango wood frame for long term durability • Double cushioned Sofa Bed to provide you with extra softness to make a fine seating experience • A double bed that can easily sleep two,Specifications of FabHomeDecor Fabric Double Sofa Bed (Finish Color - Leatherette Black Mechanism Type - Pull Out) Installation & Demo Installation & Demo Details Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product. In The Box 1 Sofa Bed General Brand FabHomeDecor Mattress Included No Delivery Condition Knock Down Storage Included No Mechanism Type Pull Out Type Sofa Bed Style Contemporary & Modern 

In [14]:
dataset['savings'] = dataset['retail_price'] - dataset['discounted_price']

In [15]:
dataset.head()

,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,description,brand,product_specifications,savings
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",620.0
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",9511.0
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",500.0
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",432.0
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",10.0


In [16]:
dataset.head()

,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,description,brand,product_specifications,savings
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",620.0
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",9511.0
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",500.0
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",432.0
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",10.0


In [17]:
dataset.iloc[0].product_category_tree

'["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

In [18]:
input_string = '["Clothing >> Women\'s Clothing >> Lingerie, Sleep & Swimwear >> Shorts >> Alisha Shorts >> Alisha Solid Women\'s Cycling Shorts"]'

cleaned_string = re.sub(r'[\\\'\"]', '', input_string)
inside_brackets = cleaned_string.strip('[]')

# Split the string using ' >> ' as the separator
word_list = inside_brackets.split(' >> ')

print(word_list)

['Clothing', 'Womens Clothing', 'Lingerie, Sleep & Swimwear', 'Shorts', 'Alisha Shorts', 'Alisha Solid Womens Cycling Shorts']


In [19]:
#works good
def clean_and_split(text):
    cleaned_string = re.sub(r'[\\\'\"]', '', text)
    inside_brackets = cleaned_string.strip('[]')
    word_list = inside_brackets.split(' >> ')
    return word_list

# Apply the cleaning and splitting function to the column
dataset['product_category_list'] = dataset['product_category_tree'].apply(clean_and_split)

# Display the modified DataFrame
dataset.head()

,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,description,brand,product_specifications,savings,product_category_list
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",620.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ..."
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",9511.0,"[Furniture, Living Room Furniture, Sofa Beds &..."
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",500.0,"[Footwear, Womens Footwear, Ballerinas, AW Bel..."
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",432.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ..."
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",10.0,"[Pet Supplies, Grooming, Skin & Coat Care, Sha..."


In [20]:
dataset.drop('product_category_tree',axis=1,inplace=True)

In [21]:
dataset.head()

,product_url,product_name,pid,retail_price,discounted_price,image,description,brand,product_specifications,savings,product_category_list
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",620.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ..."
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",9511.0,"[Furniture, Living Room Furniture, Sofa Beds &..."
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",500.0,"[Footwear, Womens Footwear, Ballerinas, AW Bel..."
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",432.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ..."
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",10.0,"[Pet Supplies, Grooming, Skin & Coat Care, Sha..."


In [22]:
bin_edges = [0, 200, 1000, 5500, 15000, float('inf')]
bin_labels = ['verylow', 'low', 'medium', 'high', 'veryhigh']

# Apply binning and create a new 'price_category' column
dataset['retail_price_category'] = pd.cut(dataset['retail_price'], bins=bin_edges, labels=bin_labels, right=False)


In [23]:
dataset.head()

,product_url,product_name,pid,retail_price,discounted_price,image,description,brand,product_specifications,savings,product_category_list,retail_price_category
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",620.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",9511.0,"[Furniture, Living Room Furniture, Sofa Beds &...",veryhigh
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",500.0,"[Footwear, Womens Footwear, Ballerinas, AW Bel...",low
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",432.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",10.0,"[Pet Supplies, Grooming, Skin & Coat Care, Sha...",low


In [24]:
bin_edges = [0, 150, 800, 4900, 13000, float('inf')]
bin_labels = ['dcodeverylow', 'dcodelow', 'dcodemedium', 'dcodehigh', 'dcodeveryhigh']

# Apply binning and create a new 'price_category' column
dataset['discounted_price_category'] = pd.cut(dataset['discounted_price'], bins=bin_edges, labels=bin_labels, right=False)


In [25]:
dataset.head()

,product_url,product_name,pid,retail_price,discounted_price,image,description,brand,product_specifications,savings,product_category_list,retail_price_category,discounted_price_category
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",620.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",9511.0,"[Furniture, Living Room Furniture, Sofa Beds &...",veryhigh,dcodeveryhigh
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",500.0,"[Footwear, Womens Footwear, Ballerinas, AW Bel...",low,dcodelow
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",432.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",10.0,"[Pet Supplies, Grooming, Skin & Coat Care, Sha...",low,dcodelow


In [26]:
dataset.drop(['retail_price','discounted_price'],axis=1,inplace=True)

In [27]:
dataset.head()

,product_url,product_name,pid,image,description,brand,product_specifications,savings,product_category_list,retail_price_category,discounted_price_category
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",620.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",9511.0,"[Furniture, Living Room Furniture, Sofa Beds &...",veryhigh,dcodeveryhigh
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",500.0,"[Footwear, Womens Footwear, Ballerinas, AW Bel...",low,dcodelow
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",432.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",10.0,"[Pet Supplies, Grooming, Skin & Coat Care, Sha...",low,dcodelow


In [28]:
bin_edges = [0, 50, 300, 1000, 4500, float('inf')]
bin_labels = ['savingsverylow', 'savingscodelow', 'savingscodemedium', 'savingscodehigh', 'savingscodeveryhigh']

# Apply binning and create a new 'price_category' column
dataset['savings_price_category'] = pd.cut(dataset['savings'], bins=bin_edges, labels=bin_labels, right=False)


In [29]:
dataset.head()

,product_url,product_name,pid,image,description,brand,product_specifications,savings,product_category_list,retail_price_category,discounted_price_category,savings_price_category
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",620.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow,savingscodemedium
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...",9511.0,"[Furniture, Living Room Furniture, Sofa Beds &...",veryhigh,dcodeveryhigh,savingscodeveryhigh
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...",500.0,"[Footwear, Womens Footwear, Ballerinas, AW Bel...",low,dcodelow,savingscodemedium
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...",432.0,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow,savingscodemedium
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...",10.0,"[Pet Supplies, Grooming, Skin & Coat Care, Sha...",low,dcodelow,savingsverylow


In [30]:
dataset.drop('savings',axis=1,inplace=True)

In [31]:
dataset.head()

,product_url,product_name,pid,image,description,brand,product_specifications,product_category_list,retail_price_category,discounted_price_category,savings_price_category
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow,savingscodemedium
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...","[Furniture, Living Room Furniture, Sofa Beds &...",veryhigh,dcodeveryhigh,savingscodeveryhigh
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...","[Footwear, Womens Footwear, Ballerinas, AW Bel...",low,dcodelow,savingscodemedium
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow,savingscodemedium
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...","[Pet Supplies, Grooming, Skin & Coat Care, Sha...",low,dcodelow,savingsverylow


In [32]:
dataset.iloc[0].product_specifications

'{"product_specification"=>[{"key"=>"Number of Contents in Sales Package", "value"=>"Pack of 3"}, {"key"=>"Fabric", "value"=>"Cotton Lycra"}, {"key"=>"Type", "value"=>"Cycling Shorts"}, {"key"=>"Pattern", "value"=>"Solid"}, {"key"=>"Ideal For", "value"=>"Women\'s"}, {"value"=>"Gentle Machine Wash in Lukewarm Water, Do Not Bleach"}, {"key"=>"Style Code", "value"=>"ALTHT_3P_21"}, {"value"=>"3 shorts"}]}'

In [33]:
dataset.iloc[1].product_specifications

'{"product_specification"=>[{"key"=>"Installation & Demo Details", "value"=>"Installation and demo for this product is done free of cost as part of this purchase. Our service partner will visit your location within 72 business hours from the delivery of the product."}, {"value"=>"1 Sofa Bed"}, {"key"=>"Brand", "value"=>"FabHomeDecor"}, {"key"=>"Mattress Included", "value"=>"No"}, {"key"=>"Delivery Condition", "value"=>"Knock Down"}, {"key"=>"Storage Included", "value"=>"No"}, {"key"=>"Mechanism Type", "value"=>"Pull Out"}, {"key"=>"Type", "value"=>"Sofa Bed"}, {"key"=>"Style", "value"=>"Contemporary and Modern"}, {"key"=>"Filling Material", "value"=>"Microfiber"}, {"key"=>"Seating Capacity", "value"=>"3 Seater"}, {"key"=>"Upholstery Type", "value"=>"NA"}, {"key"=>"Upholstery Included", "value"=>"No"}, {"key"=>"Bed Size", "value"=>"Double"}, {"key"=>"Shape", "value"=>"Square"}, {"key"=>"Suitable For", "value"=>"Living Room"}, {"key"=>"Model Number", "value"=>"FHD112"}, {"key"=>"Care Ins

In [34]:
#specification later

In [35]:
dataset.head()

,product_url,product_name,pid,image,description,brand,product_specifications,product_category_list,retail_price_category,discounted_price_category,savings_price_category
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow,savingscodemedium
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati...","[Furniture, Living Room Furniture, Sofa Beds &...",veryhigh,dcodeveryhigh,savingscodeveryhigh
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"{""product_specification""=>[{""key""=>""Ideal For""...","[Footwear, Womens Footwear, Ballerinas, AW Bel...",low,dcodelow,savingscodemedium
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"{""product_specification""=>[{""key""=>""Number of ...","[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow,savingscodemedium
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",...","[Pet Supplies, Grooming, Skin & Coat Care, Sha...",low,dcodelow,savingsverylow


In [36]:
# Function to extract values and remove spaces
def extract_values(row):
    values = re.findall(r'"value"=>"(.*?)"', row)
    return ' , '.join(values)

# Apply the function to each row of 'product_specification'
dataset['extracted_values'] = dataset['product_specifications'].apply(extract_values)

In [37]:
dataset['extracted_values'].iloc[0]

"Pack of 3 , Cotton Lycra , Cycling Shorts , Solid , Women's , Gentle Machine Wash in Lukewarm Water, Do Not Bleach , ALTHT_3P_21 , 3 shorts"

In [38]:
dataset.drop('product_specifications',axis=1,inplace=True)

In [39]:
dataset.head()

,product_url,product_name,pid,image,description,brand,product_category_list,retail_price_category,discounted_price_category,savings_price_category,extracted_values
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow,savingscodemedium,"Pack of 3 , Cotton Lycra , Cycling Shorts , So..."
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,"[Furniture, Living Room Furniture, Sofa Beds &...",veryhigh,dcodeveryhigh,savingscodeveryhigh,Installation and demo for this product is done...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,"[Footwear, Womens Footwear, Ballerinas, AW Bel...",low,dcodelow,savingscodemedium,"Women , Casual , Red , Patent Leather , 1 inch..."
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,"[Clothing, Womens Clothing, Lingerie, Sleep & ...",low,dcodelow,savingscodemedium,"Pack of 2 , Cotton Lycra , Cycling Shorts , So..."
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,"[Pet Supplies, Grooming, Skin & Coat Care, Sha...",low,dcodelow,savingsverylow,"Dog , Sicons , 500 ml , SH.DF-14 , All Purpose..."


In [40]:
def list_to_string(lst):
    return ' '.join(lst)

# Apply the function to the 'product_category_list' column
dataset['product_category_string'] = dataset['product_category_list'].apply(list_to_string)

In [41]:
dataset.drop('product_category_list',axis=1,inplace=True)

In [42]:
dataset.head()

,product_url,product_name,pid,image,description,brand,retail_price_category,discounted_price_category,savings_price_category,extracted_values,product_category_string
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,low,dcodelow,savingscodemedium,"Pack of 3 , Cotton Lycra , Cycling Shorts , So...","Clothing Womens Clothing Lingerie, Sleep & Swi..."
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,veryhigh,dcodeveryhigh,savingscodeveryhigh,Installation and demo for this product is done...,Furniture Living Room Furniture Sofa Beds & Fu...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,low,dcodelow,savingscodemedium,"Women , Casual , Red , Patent Leather , 1 inch...",Footwear Womens Footwear Ballerinas AW Bellies
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,low,dcodelow,savingscodemedium,"Pack of 2 , Cotton Lycra , Cycling Shorts , So...","Clothing Womens Clothing Lingerie, Sleep & Swi..."
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,low,dcodelow,savingsverylow,"Dog , Sicons , 500 ml , SH.DF-14 , All Purpose...",Pet Supplies Grooming Skin & Coat Care Shampoo...


In [43]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14058 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype   
---  ------                     --------------  -----   
 0   product_url                14058 non-null  object  
 1   product_name               14058 non-null  object  
 2   pid                        14058 non-null  object  
 3   image                      14058 non-null  object  
 4   description                14058 non-null  object  
 5   brand                      14058 non-null  object  
 6   retail_price_category      14058 non-null  category
 7   discounted_price_category  14058 non-null  category
 8   savings_price_category     14058 non-null  category
 9   extracted_values           14058 non-null  object  
 10  product_category_string    14058 non-null  object  
dtypes: category(3), object(8)
memory usage: 1.0+ MB


In [44]:
categorical_columns = ['product_name','description','brand','product_category_string','retail_price_category','savings_price_category','discounted_price_category','extracted_values']
for col in categorical_columns:
    dataset[col] = dataset[col].astype(str)

In [45]:
dataset.info()
# Check data types of all columns
print(dataset.dtypes)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14058 entries, 0 to 19999
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   product_url                14058 non-null  object
 1   product_name               14058 non-null  object
 2   pid                        14058 non-null  object
 3   image                      14058 non-null  object
 4   description                14058 non-null  object
 5   brand                      14058 non-null  object
 6   retail_price_category      14058 non-null  object
 7   discounted_price_category  14058 non-null  object
 8   savings_price_category     14058 non-null  object
 9   extracted_values           14058 non-null  object
 10  product_category_string    14058 non-null  object
dtypes: object(11)
memory usage: 1.3+ MB
product_url                  object
product_name                 object
pid                          object
image                      

In [46]:
dataset.head()

,product_url,product_name,pid,image,description,brand,retail_price_category,discounted_price_category,savings_price_category,extracted_values,product_category_string
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,low,dcodelow,savingscodemedium,"Pack of 3 , Cotton Lycra , Cycling Shorts , So...","Clothing Womens Clothing Lingerie, Sleep & Swi..."
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,veryhigh,dcodeveryhigh,savingscodeveryhigh,Installation and demo for this product is done...,Furniture Living Room Furniture Sofa Beds & Fu...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,low,dcodelow,savingscodemedium,"Women , Casual , Red , Patent Leather , 1 inch...",Footwear Womens Footwear Ballerinas AW Bellies
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,low,dcodelow,savingscodemedium,"Pack of 2 , Cotton Lycra , Cycling Shorts , So...","Clothing Womens Clothing Lingerie, Sleep & Swi..."
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,low,dcodelow,savingsverylow,"Dog , Sicons , 500 ml , SH.DF-14 , All Purpose...",Pet Supplies Grooming Skin & Coat Care Shampoo...


In [47]:
dataset['tags']=dataset['product_name']+dataset['description']+dataset['brand']+dataset['product_category_string']+dataset['retail_price_category']+dataset['savings_price_category']+dataset['discounted_price_category']+dataset['extracted_values']+dataset['brand']

In [48]:
dataset.head()

,product_url,product_name,pid,image,description,brand,retail_price_category,discounted_price_category,savings_price_category,extracted_values,product_category_string,tags
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,low,dcodelow,savingscodemedium,"Pack of 3 , Cotton Lycra , Cycling Shorts , So...","Clothing Womens Clothing Lingerie, Sleep & Swi...",Alisha Solid Women's Cycling ShortsKey Feature...
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor Fabric Double Sofa Bed (Finish Co...,FabHomeDecor,veryhigh,dcodeveryhigh,savingscodeveryhigh,Installation and demo for this product is done...,Furniture Living Room Furniture Sofa Beds & Fu...,FabHomeDecor Fabric Double Sofa BedFabHomeDeco...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",Key Features of AW Bellies Sandals Wedges Heel...,AW,low,dcodelow,savingscodemedium,"Women , Casual , Red , Patent Leather , 1 inch...",Footwear Womens Footwear Ballerinas AW Bellies,AW BelliesKey Features of AW Bellies Sandals W...
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Key Features of Alisha Solid Women's Cycling S...,Alisha,low,dcodelow,savingscodemedium,"Pack of 2 , Cotton Lycra , Cycling Shorts , So...","Clothing Womens Clothing Lingerie, Sleep & Swi...",Alisha Solid Women's Cycling ShortsKey Feature...
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Specifications of Sicons All Purpose Arnica Do...,Sicons,low,dcodelow,savingsverylow,"Dog , Sicons , 500 ml , SH.DF-14 , All Purpose...",Pet Supplies Grooming Skin & Coat Care Shampoo...,Sicons All Purpose Arnica Dog ShampooSpecifica...


In [49]:
dataset.drop(['description','retail_price_category','discounted_price_category','savings_price_category','extracted_values','product_category_string'],axis=1,inplace=True)

In [50]:
dataset.head()

,product_url,product_name,pid,image,brand,tags
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Alisha,Alisha Solid Women's Cycling ShortsKey Feature...
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor,FabHomeDecor Fabric Double Sofa BedFabHomeDeco...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",AW,AW BelliesKey Features of AW Bellies Sandals W...
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Alisha,Alisha Solid Women's Cycling ShortsKey Feature...
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Sicons,Sicons All Purpose Arnica Dog ShampooSpecifica...


In [51]:
dataset.iloc[0].tags

"Alisha Solid Women's Cycling ShortsKey Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shortsAlishaClothing Womens Clothing Lingerie, Sleep & Swimwear Shorts Alisha Shorts Alisha Solid Womens Cycling ShortslowsavingscodemediumdcodelowPack of 3 , Cotton Lycra , Cycling Shorts , Solid , Women's , Gentle Machine Wash in Lukewarm Water, Do Not Bleach , ALTHT_3P_21 , 3 shortsAlisha"

In [52]:
# Split the input string using a regular expression to capture the desired values
values = re.findall(r'\b[A-Z][A-Z\s,]+(?=\b[A-Z])', input_string)

# Convert the list of values to a comma-separated string
tags = ', '.join(values)

In [53]:
dataset.head()

,product_url,product_name,pid,image,brand,tags
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Alisha,Alisha Solid Women's Cycling ShortsKey Feature...
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor,FabHomeDecor Fabric Double Sofa BedFabHomeDeco...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",AW,AW BelliesKey Features of AW Bellies Sandals W...
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Alisha,Alisha Solid Women's Cycling ShortsKey Feature...
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Sicons,Sicons All Purpose Arnica Dog ShampooSpecifica...


In [54]:
dataset.iloc[0].tags

"Alisha Solid Women's Cycling ShortsKey Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shortsAlishaClothing Womens Clothing Lingerie, Sleep & Swimwear Shorts Alisha Shorts Alisha Solid Womens Cycling ShortslowsavingscodemediumdcodelowPack of 3 , Cotton Lycra , Cycling Shorts , Solid , Women's , Gentle Machine Wash in Lukewarm Water, Do Not Bleach , ALTHT_3P_21 , 3 shortsAlisha"

In [55]:
def tokenize_and_join(text):
    tokens = re.findall(r'\w+', text)
    return ' '.join(tokens)

In [56]:
# Apply the function to the 'tags' column
dataset['tags'] = dataset['tags'].apply(tokenize_and_join)

In [57]:
dataset.iloc[0].tags

'Alisha Solid Women s Cycling ShortsKey Features of Alisha Solid Women s Cycling Shorts Cotton Lycra Navy Red Navy Specifications of Alisha Solid Women s Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women s Fabric Care Gentle Machine Wash in Lukewarm Water Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shortsAlishaClothing Womens Clothing Lingerie Sleep Swimwear Shorts Alisha Shorts Alisha Solid Womens Cycling ShortslowsavingscodemediumdcodelowPack of 3 Cotton Lycra Cycling Shorts Solid Women s Gentle Machine Wash in Lukewarm Water Do Not Bleach ALTHT_3P_21 3 shortsAlisha'

In [58]:
dataset.head()

,product_url,product_name,pid,image,brand,tags
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Alisha,Alisha Solid Women s Cycling ShortsKey Feature...
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor,FabHomeDecor Fabric Double Sofa BedFabHomeDeco...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",AW,AW BelliesKey Features of AW Bellies Sandals W...
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Alisha,Alisha Solid Women s Cycling ShortsKey Feature...
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Sicons,Sicons All Purpose Arnica Dog ShampooSpecifica...


In [59]:
dataset['tags']=dataset['tags'].apply(lambda x:x.lower())

In [60]:
dataset.head()

,product_url,product_name,pid,image,brand,tags
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Alisha,alisha solid women s cycling shortskey feature...
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor,fabhomedecor fabric double sofa bedfabhomedeco...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",AW,aw bellieskey features of aw bellies sandals w...
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Alisha,alisha solid women s cycling shortskey feature...
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Sicons,sicons all purpose arnica dog shampoospecifica...


In [61]:
dataset.head()

,product_url,product_name,pid,image,brand,tags
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Alisha,alisha solid women s cycling shortskey feature...
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor,fabhomedecor fabric double sofa bedfabhomedeco...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",AW,aw bellieskey features of aw bellies sandals w...
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Alisha,alisha solid women s cycling shortskey feature...
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Sicons,sicons all purpose arnica dog shampoospecifica...


In [62]:
ps=PorterStemmer()

In [63]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)  

In [64]:
dataset['tags']=dataset['tags'].apply(stem)

In [65]:

from sklearn.feature_extraction.text import TfidfVectorizer


In [66]:
tfv = TfidfVectorizer(max_features=None,
                     analyzer='word',
                     token_pattern=r'\w{1,}',
                     ngram_range=(1,4),
                     stop_words='english')


In [67]:
vectors=tfv.fit_transform(dataset['tags'])

In [68]:
vectors

<14058x1181610 sparse matrix of type '<class 'numpy.float64'>'
	with 4057274 stored elements in Compressed Sparse Row format>

In [69]:
from sklearn.metrics.pairwise import sigmoid_kernel

In [70]:
sig = sigmoid_kernel(vectors,vectors)

In [71]:
indices = pd.Series(dataset.index,index=dataset['product_name']).drop_duplicates()

In [72]:
indices.head(10)

product_name
Alisha Solid Women's Cycling Shorts                                      0
FabHomeDecor Fabric Double Sofa Bed                                      1
AW Bellies                                                               2
Alisha Solid Women's Cycling Shorts                                      3
Sicons All Purpose Arnica Dog Shampoo                                    4
Eternal Gandhi Super Series Crystal Paper Weights  with Silver Finish    5
Alisha Solid Women's Cycling Shorts                                      6
FabHomeDecor Fabric Double Sofa Bed                                      7
dilli bazaaar Bellies, Corporate Casuals, Casuals                        8
Alisha Solid Women's Cycling Shorts                                      9
dtype: int64

In [73]:
def product_recommendation(title,sig=sig):
    indx = indices[title]
    
    
    sig_scores = list(enumerate(sig[indx]))
    
    #sorting products
    sig_scores = sorted(sig_scores, key=lambda x: x[1], reverse=True)
    
   
    sig_scores = sig_scores[1:11]
    
    
    product_indices = [i[0] for i in sig_scores]
    
    
    return dataset['product_name'].iloc[product_indices]

In [74]:
# def recommend(product):
#     index = dataset[dataset['product_name'] == product].index[0]
#     distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
#     for i in distances[1:11]:
#         print(dataset.iloc[i[0]].product_name)

In [75]:
dataset.head()

,product_url,product_name,pid,image,brand,tags
0,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2FF9KEDEFGF,"[""http://img5a.flixcart.com/image/short/u/4/a/...",Alisha,alisha solid women s cycl shortskey featur of ...
1,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,SBEEH3QGU7MFYJFY,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",FabHomeDecor,fabhomedecor fabric doubl sofa bedfabhomedecor...
2,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,SHOEH4GRSUBJGZXE,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",AW,aw bellieskey featur of aw belli sandal wedg h...
3,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,SRTEH2F6HUZMQ6SJ,"[""http://img5a.flixcart.com/image/short/6/2/h/...",Alisha,alisha solid women s cycl shortskey featur of ...
4,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,PSOEH3ZYDMSYARJ5,"[""http://img5a.flixcart.com/image/pet-shampoo/...",Sicons,sicon all purpos arnica dog shampoospecif of s...


In [76]:
n=input("Enter the name of the product: ")
print("\nTop Recommended products are: \n")
print(product_recommendation(n).unique())

Enter the name of the product: AW Bellies

Top Recommended products are: 

["Shopnetix Stylish & Comfort Men's Lace up Mocassins (SNX-4449) Casuals"
 'Sonaxo Men Running Shoes' 'Glitzy Galz Women Heels'
 'Glitzy Galz Girls Flats' 'Port NuWave Football Shoes'
 'Lee Parke Running Shoes' 'SoleThreads Flip Flops'
 'Lee Parke Walking Shoes' 'keco Women Wedges']


In [77]:
# import pickle
# pickle.dump(dataset,open('dataset.pkl','wb'))
# pickle.dump(similarity,open('similarity.pkl','wb'))


In [78]:
pickle.dump(dataset,open('dataset.pkl','wb'))
pickle.dump(sig,open('sig.pkl','wb'))